# Segmenting X-Ray Images using Neural Networks

The aim of this notebook to walk through the process of performing inference on a pretrianed model to segment X-Ray images into bone, soft-tissue and open beam regions

## Download data and model

In [ ]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1Wel_XsyE7HcEq0TkZWI61GABO4jOtj9C',
                                    dest_path='./dataset.hdf5')
gdd.download_file_from_google_drive(file_id='1cePD5E-T9mr5W0xPGuzEnUt8Glpvn23U',
                                    dest_path='./model.h5')

In [ ]:
import os, sys
import numpy as np
import h5py
import matplotlib.pyplot as plt

#import Keras sub-modules
from keras.models import Model #functional API for Keras (best for greater flexibility)
from keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense #'main' layers
from keras.layers import BatchNormalization, Dropout #regulartisation layers
from keras.layers import Activation
from keras.optimizers import * #import all optimisers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, CSVLogger #callbacks for model performance analysis
from keras.metrics import categorical_accuracy #metrics for model performance
from keras import backend as K #gives backend functionality
from keras import losses #imports pre-defined loss functions
from keras.models import load_model #allows pre-trained models to be called back

g5 files compress the data for convenient storage, as well as offering accessibility through a 'key' system. This is a useful file type, however, if it can become cumbersome for large datasets. Other storage mechanisms will still work with this model.

In [ ]:
hdf5_path = "./dataset.hdf5" ## this is our h5 file containing training and testing data
dataset = h5py.File(hdf5_path , 'r')

classes = 3

test_images = dataset['test_img'][:]
no_images, height, width, channels = test_images.shape

test_labels =dataset['test_label'][:].reshape(-1,height*width, classes )

dataset.close()

## Load model

In [ ]:
model = load_model("./model.h5")

The above should have been relatively straight forward. We simply use the existing data files we have created, called them, and compiled the model based on this input. Before progressing further, we shall now investigate the model. By printing a model summary, Keras provides a user friendly output which shows the layers and their parameters.

In [ ]:
model.summary()

With this summary in mind, go to the ```XNet.py``` script and have a look.

The process of training and saving a model is summarised in the ```Training/TrainingClass.py``` script. Have a look at that script to see what the different functions are doing. In the class you will see that all the augementations are being performed on the data, and so you just need to create an HDF5 file which contains non-augmented images and call it into the training class.

Once you are happy with your understanding of the training class we shall now try predicting from the model. In Keras this is very simple. Pass in the test image into the ```model.predict``` function and reshape the output to view it.

Choose an image to test on

In [ ]:
test_index = 0

In [ ]:
testing_image = test_images[test_index]

#as we are only running one image, we must reshape to shape (batch, height, width, channels)
testing_image = testing_image.reshape((1,200,200,1))

In [ ]:
prediction = model.predict(testing_image)

#the prediction is a flattened array and so must be reshaped.
#there are 3 channels as we are actually outputting the probability map over all 3 classes.
prediction = prediction.reshape((200,200,3))

In [ ]:
plt.imshow(prediction)

After performing prediction various postprocessing stages can be employed to fine tune the output. See the ```PostProcessing.py``` script for more details, which is then called in the ```Training/train.py``` script.